In [1]:
import fnmatch
import cv2
import numpy as np
import string
import time

from keras_preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image,ImageFont,ImageDraw
from sklearn.utils import shuffle

import gradio as gr
import imgaug.augmenters as iaa

from SegmentPage import segment_into_lines
from SegmentLine import segment_into_words
from RecognizeWord import recognize_words

C:\Users\h.decure\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## GENERATE

In [2]:
def generate_data(n_samples = 150000, word_type = 'all', progress=gr.Progress()):
    global gray_back
    kernel=np.ones((2,2),np.uint8)
    kernel2=np.ones((1,1),np.uint8)
    punclist=',.?!:;'

    #Character sets to choose from.
    smallletters=string.ascii_lowercase
    capitalletters=string.ascii_uppercase
    digits=string.digits
    all_char = smallletters+capitalletters+digits+punclist
    
    #Base backgound.
    backfilelist=os.listdir('./background/')
    backgroud_list=[]

    for bn in backfilelist:
        fileloc='./background/'+bn
        backgroud_list.append(Image.fromarray(cv2.imread(fileloc,0)))


    #Different fonts to be used.
    fonts_list=os.listdir('./fonts/')
    fonts_list=['./fonts/'+f for f in fonts_list]

    #Lengths of the words.
    word_lengths=[]
    for l in range(1,21):
        word_lengths.append(l)

    #Font size.
    font_size=[]
    for l in range(10,30):
        font_size.append(l)


    file_counter=0


    def random_brightness(img):
        img=np.array(img)
        brightness=iaa.Multiply((0.2,1.2))
        img=brightness.augment_image(img)
        return img

    def dilation(img):
        img=np.array(img)
        img=cv2.dilate(img,kernel2,iterations=1)
        return img

    def erosion(img):
        img=np.array(img)
        img=cv2.erode(img,kernel,iterations=1)
        return img

    def blur(img):
        img=np.array(img)
        img=cv2.blur(img,ksize=(3,3))



    def fuse_gray(img):
        img=np.array(img)
        ht,wt=img.shape[0],img.shape[1]
        gray_back=cv2.imread('gray_back.jpg',0)
        gray_back=cv2.resize(gray_back,(wt,ht))

        blended=cv2.addWeighted(src1=img,alpha=0.8,src2=gray_back,beta=0.4,gamma=10)
        return blended

    def random_transformation(img):
        if np.random.rand()<0.5:
            img=fuse_gray(img)
        elif np.random.rand()<0.5:
            img=random_brightness(img)
        elif np.random.rand()<0.5:
            img=dilation(img)
        elif np.random.rand()<0.5:
            img=erosion(img)

        else:
            img=np.array(img)
        return Image.fromarray(img)

    file=open('annotation.txt','a+')

    file_counter=0
    progress(0, desc="Starting...")
    for i in progress.tqdm(range(int(n_samples))):
        back_c=random.choice(backgroud_list).copy()
        start_cap=random.choice(capitalletters)
        filename=''.join([random.choice(smallletters) for c in range(random.choice([5,6,7,8,9,10,11]))])
        random_font=random.choice(fonts_list)
        try :
            font=ImageFont.truetype(random_font,size=random.choice(font_size))
        except :
            print(random_font)
        if word_type=='all':
            word=''.join([random.choice(all_char) for b in range(random.choice(word_lengths))])
        elif word_type=='lowercase':
            word=''.join([random.choice(smallletters) for b in range(random.choice(word_lengths))])
        elif word_type=='uppercase':
            word=''.join([random.choice(capitalletters) for b in range(random.choice(word_lengths))])
        elif word_type=='firstcapital':
            word=''.join([random.choice(smallletters) for b in range(random.choice(word_lengths)-1)])
            word=start_cap+word
        elif word_type=='digits':
            word=''.join([random.choice(digits) for b in range(random.choice(word_lengths))])
        elif word_type=='punctuation':
            word=''.join([random.choice(smallletters) for b in range(random.choice(word_lengths))])
            word=word+str(random.choice(punclist))
        else:
            raise Exception("Invalid word choice.")
            
        left, top, right, bottom = font.getbbox(word) 
        w = right - left
        h = bottom  
        
        back_c=back_c.resize((w+5,h+5))
        draw=ImageDraw.Draw(back_c)
        draw.text((0,0),text=word,font=font,fill='rgb(0,0,0)')
        back_c=random_transformation(back_c)
        back_c.save(f'./images/{file_counter}_{filename}.jpg')
        #print(f'./images/{file_counter}_{filename}.jpg   ', random_font)
        file.writelines(str(file_counter)+'_'+filename+'.jpg'+'\t'+word+'\n')
        file_counter+=1
    return "Done !"

## Train CRNN

In [3]:
# Crée une liste des lettres et chiffres existants
punclist=',.?!:;'
char_list = string.ascii_letters+string.digits+punclist

# Transforme les mots en liste de chiffres  
def encode_to_labels(txt):
    # encoding each output word into digits
    # Crée une liste pour l'output
    dig_lst = []
    # Pour chaque caractère dans le mot :
    for index, char in enumerate(txt):
        try:
            # Ajouter dans la liste "dig_lst" le numéro d'index du caractère
            dig_lst.append(char_list.index(char))
        except:
            print(char)       
    return dig_lst

# Trouve la couleur dominante d'une image
def find_dominant_color(image):
        # Resizing parameters
        width, height = 150,150
        # Fait une interpolation pour changer la taille de l'image source
        image = image.resize((width, height),resample = 0)
        # Get colors from image object
        pixels = image.getcolors(width * height)
        # Sort them by count number(first element of tuple)
        sorted_pixels = sorted(pixels, key=lambda t: t[0])
        # Get the most frequent color
        dominant_color = sorted_pixels[-1][1]
        return dominant_color

def preprocess_img(img, imgSize):
    "put img into target img of size imgSize, transpose for TF and normalize gray-values"

    # there are damaged files in IAM dataset - just use black image instead
    if img is None:
        img = np.zeros([imgSize[1], imgSize[0]]) 
        print("Image None!")

    # create target image and copy sample image into it
    # Récupère la largeur et hauteur attendue "wt" pour Width Target et "ht" pour Height Target
    (wt, ht) = imgSize
    # Récupère la hauteur et largeur de l'image source
    (h, w) = img.shape
    # Fait le rapport de la largeur sur la largeur attendue
    fx = w / wt
    # Fait le rapport de la hauteur sur la hauteur attendue
    fy = h / ht
    # Prend le rapport le plus grand des deux
    f = max(fx, fy)
    # Définie la nouvelle taille de l'image en prennant le plus grand entre :
    # - Le plus petit entre la largeur attendue et la largeur actuelle divisée par le rapport f
    # - 1
    newSize = (max(min(wt,
                       int(w / f)), 
                   1),
               max(min(ht,
                       int(h / f)),
                   1)
              )  
    # scale according to f (result at least 1 and at most wt or ht)
    # Fait une interpolation pour changer la taille de l'image vers la nouvelle taille
    img = cv2.resize(img, newSize, interpolation=cv2.INTER_CUBIC) # INTER_CUBIC interpolation best approximate the pixels image
                                                                  # see this https://stackoverflow.com/a/57503843/7338066

    # Trouve la couleur dominante
    most_freq_pixel=find_dominant_color(Image.fromarray(img))
    # Fait un applat de la couleur dominante
    target = np.ones([ht, wt]) * most_freq_pixel
    # Fusionner l'applat et l'image dont la taille a été changée pour faire de l'érodage
    target[0:newSize[1], 0:newSize[0]] = img

    img = target
    return img

def CRNN_train(batch_size = 256,
               epochs = 15,
               learning_rate = 0.0001,
               split = 10,
               decay = True,
               nbr_smpl_train = 150000,
               progress=gr.Progress()
              ):
    batch_size = int(batch_size)
    epochs = int(epochs)
    learning_rate = float(learning_rate)
    split = int(split)
    nbr_smpl = int(nbr_smpl_train)
    #lists for training dataset
    training_img = []
    training_txt = []
    train_input_length = []
    train_label_length = []
    orig_txt = []

    #lists for validation dataset
    valid_img = []
    valid_txt = []
    valid_input_length = []
    valid_label_length = []
    valid_orig_txt = []

    max_label_len = 0

    # Ouvre le fichier annotation.txt qui contient le nom des images , le contenu des images
    annot=open('./annotation.txt','r').readlines()
    imagenames=[]
    txts=[]

    # Récupère les données dans annotation.txt et répartis en deux listes
    for cnt in annot:
        filename,txt=cnt.split('\t')[0],cnt.split('\t')[1].split('\n')[0]
        imagenames.append(filename)
        txts.append(txt)


    # Mélange les données pour un seed différent à chaque fois
    c = list(zip(imagenames, txts))
    random.shuffle(c)
    imagenames, txts = zip(*c)


    # Pour chaque image :
    #for i in progress.tqdm(range(len(imagenames))):
    for i in progress.tqdm(range(nbr_smpl)):
            # Lis l'image
            img = cv2.imread('./images/'+imagenames[i],0)   
            # Effectue le preprocess sur l'image
            img=preprocess_img(img,(128,32))
            # Augmente la profondeur de l'array d'un niveau. [a] -> [[a]]
            img=np.expand_dims(img,axis=-1)
            # Diviser les valeurs des pixels par 255 pour avoir une representation 0-1
            img = img/255
            # Récupere la légende correspondante
            txt = txts[i]

            # Compute maximum length of the text
            if len(txt) > max_label_len:
                max_label_len = len(txt)


            # Split the 150000 data into validation and training dataset as 10% and 90% respectively
            if i%split == 0:     
                valid_orig_txt.append(txt)   
                valid_label_length.append(len(txt))
                valid_input_length.append(31)
                valid_img.append(img)
                valid_txt.append(encode_to_labels(txt))
            else:
                orig_txt.append(txt)   
                train_label_length.append(len(txt))
                train_input_length.append(31)
                training_img.append(img)
                training_txt.append(encode_to_labels(txt)) 

            # Break the loop if total data is nbr_smpl
            if i == nbr_smpl:
                flag = 1
                break
            i+=1

    # Pad each output label to maximum text length
    # Ajoute un chiffre non-utilisé dans la liste de caractères derrière les légendes pour qu'elles aient toutes la taille maximale
    train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
    valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

    inputs = Input(shape=(32,128,1))

    # convolution layer with kernel size (3,3)
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
    # "relu" = max(0,x)
    conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
    # poolig layer with kernel size (2,2)
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
    pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

    conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
    pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

    conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)

    conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
    # poolig layer with kernel size (2,1)
    pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)

    conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)

    # Batch normalization layer
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
    batch_norm_5 = BatchNormalization()(conv_5)

    conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
    batch_norm_6 = BatchNormalization()(conv_6)
    pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

    conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)

    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Lambda 
    squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

    # bidirectional LSTM layers with units=128
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional
    blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
    blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)

    outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

    # model to be used at test time
    act_model = Model(inputs, outputs)

    labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')

    def ctc_lambda_func(args):
        y_pred, labels, input_length, label_length = args

        return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

    #model to be used at training time
    model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)
    
    if decay == True :
        initial_learning_rate = learning_rate
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate,
            decay_steps=100000,
            decay_rate=0.96,
            staircase=True)
    else :
        lr_schedule = learning_rate
        
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule), run_eagerly=True)
    
    filepath="./best_model.hdf5"
    checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    callbacks_list = [checkpoint]

    # Conversion en array
    training_img = np.array(training_img)
    train_input_length = np.array(train_input_length)
    train_label_length = np.array(train_label_length)

    valid_img = np.array(valid_img)
    valid_input_length = np.array(valid_input_length)
    valid_label_length = np.array(valid_label_length)
                                 
    yield "Génération de modèle en cours..."
    model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length], y=np.zeros(len(training_img)), batch_size=batch_size, epochs = epochs, validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]), verbose = 1, callbacks = callbacks_list)
    return "Done !"

## Evaluate

In [4]:
def evaluate(model_input, 
             nbr_smpl_eval, 
             progress=gr.Progress()
            ):
    
    nbr_smpl = int(nbr_smpl_eval)
    inputs = Input(shape=(32,128,1))
    conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
    pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
    conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
    pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
    conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
    conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
    pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
    conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
    batch_norm_5 = BatchNormalization()(conv_5)
    conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
    batch_norm_6 = BatchNormalization()(conv_6)
    pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
    conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
    squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

    blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
    blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)

    outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)
    
    #lists for training dataset
    training_img = []
    training_txt = []
    train_input_length = []
    train_label_length = []
    orig_txt = []

    max_label_len = 0

    # Ouvre le fichier annotation.txt qui contient le nom des images , le contenu des images
    annot=open('./annotation.txt','r').readlines()
    imagenames=[]
    txts=[]

    # Récupère les données dans annotation.txt et répartis en deux listes
    for cnt in annot:
        filename,txt=cnt.split(',')[0],cnt.split(',')[1].split('\n')[0]
        imagenames.append(filename)
        txts.append(txt)

    # Mélange les données pour un seed différent à chaque fois
    c = list(zip(imagenames, txts))
    random.shuffle(c)
    imagenames, txts = zip(*c)
    # Pour chaque image :
    for i in progress.tqdm(range(nbr_smpl)):
        # Lis l'image
        img = cv2.imread('./images/'+imagenames[i],0)   
        # Effectue le preprocess sur l'image
        img=preprocess_img(img,(128,32))
        # Augmente la profondeur de l'array d'un niveau. [a] -> [[a]]
        img=np.expand_dims(img,axis=-1)
        # Diviser les valeurs des pixels par 255 pour avoir une representation 0-1
        img = img/255
        # Récupere la légende correspondante
        txt = txts[i]

        # Compute maximum length of the text
        if len(txt) > max_label_len:
            max_label_len = len(txt)

        orig_txt.append(txt)   
        train_label_length.append(len(txt))
        train_input_length.append(31)
        training_img.append(img)
        training_txt.append(encode_to_labels(txt)) 
    
    labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    
    def ctc_lambda_func(args):
        y_pred, labels, input_length, label_length = args
        return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])
    act_model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)
    act_model.load_weights(model_input.orig_name)
    
    training_img = np.array(training_img)
    train_input_length = np.array(train_input_length)
    train_label_length = np.array(train_label_length)

    train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
    # loss="kullback_leibler_divergence"
    yield "Calcul en cours..."
    act_model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')
    x=[training_img, train_padded_txt, train_input_length, train_label_length]
    y=np.zeros(len(training_img))
    scores = act_model.evaluate(x, y, verbose=1)

    return "%s : %.2f" % (act_model.metrics_names[0], scores)

## Recognize

In [5]:
def recognize(img,language):
    #Open image and segment into lines
    try : 
        img = Image.open(img)
    except:
        img = img
    img = img.save("temp.jpg")
    
    line_img_array=segment_into_lines("temp.jpg")
    
    #Creating lists to store the line indexes,words list.
    full_index_indicator=[]
    all_words_list=[]
    #Variable to count the total no of lines in page.
    len_line_arr=0
    
    yield "Détection en cours..."
    #Segment the lines into words and store as arrays.
    for idx,im in enumerate(line_img_array):
        line_indicator,word_array=segment_into_words(im,idx)
        for k in range(len(word_array)):
            full_index_indicator.append(line_indicator[k])
            all_words_list.append(word_array[k])
        len_line_arr+=1

    all_words_list=np.array(all_words_list)

    yield "Reconnaissance des mots en cours..."
    #Perform the recognition on list of list of words.
    recognize_words(full_index_indicator,all_words_list,len_line_arr,language)
    
    file=open('recognized_texts.txt','r')
    pred=[]
    for i in file:
        pred.append(i)
    yield pred
    return

## Interface

In [12]:
with gr.Blocks() as demo:
    gr.Markdown("OCR From Scratch")
    
    with gr.Tab("Generate Data"):
        n_samples = gr.Number(value=150000)
        word_type = gr.Radio(
        ['all','lowercase', 'uppercase', 'firstcapital', 'digits', 'punctuation'], label="Type" , value = 'all'
        )
        generate_button = gr.Button("GENERATE")
        generate_output = gr.Textbox(show_label=False,interactive=False)
        
    with gr.Tab("Train CRNN"):
        batch_size = gr.Number(value=256, label="batch_size")
        epochs = gr.Number(value=15, label="epochs")
        learning_rate = gr.Slider(0.000000000001, 0.0001, value=0.0001, step=0.000000000001, label = "Learning Rate")
        decay = gr.Checkbox(value=True, label="Activer le Decay du Learning Rate")
        split = gr.Slider(0, 100, value=10, interactive=True, label = "Pourcentage du Dataset à utiliser pour Test")
        maximum = 0
        for path in os.listdir("./images/"):
                maximum += 1
        nbr_smpl_train = gr.Slider(0, maximum, value=maximum, interactive=True, step=100, label = "Nombres d'images du Dataset à utiliser")
        CRNN_button = gr.Button("TRAIN")
        train_text_output = gr.Textbox(show_label=False,interactive=False)
        
    with gr.Tab("Evaluate"):
        model_input = gr.File(label="Modèle à évaluer")
        maximum = 0
        for path in os.listdir("./images/"):
                maximum += 1
        nbr_smpl_eval = gr.Slider(0, maximum, value=maximum, interactive=True, step=100, label = "Nombres d'images du Dataset à utiliser")
        evaluate_button = gr.Button("Evaluate")
        eval_text_output = gr.Textbox(show_label=False,interactive=False)
    
    with gr.Tab("Recognize"):
        image_input = gr.Image(type="filepath")
        language = gr.Radio(['en','fr','es','de','pt'], label = 'Language', value = 'en', interactive=True)
        recognize_button = gr.Button("Recognize")
        recon_text_output = gr.Textbox(show_label=False,interactive=False)
    
    with gr.Tab("Live"):
        with gr.Row():
            cam_input = gr.Image(source="webcam", mirror_webcam=False, type = "pil")
            cam_text_output = gr.Textbox(show_label=False,interactive=False)
        language = gr.Radio(['en','fr','es','de','pt'], label = 'Language', value = 'en', interactive=True)
        live_button = gr.Button("Recognize")
      
    generate_button.click(generate_data, inputs=[n_samples, word_type], outputs=generate_output)
    CRNN_button.click(CRNN_train, inputs=[batch_size, epochs, learning_rate, decay, split, nbr_smpl_train], outputs=train_text_output)
    evaluate_button.click(evaluate, inputs=[model_input, nbr_smpl_eval], outputs=eval_text_output)
    recognize_button.click(recognize, inputs=[image_input,language], outputs=recon_text_output)
    live_button.click(recognize, inputs=[cam_input,language], outputs=cam_text_output)

demo.queue().launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Epoch 1/250


Traceback (most recent call last):
  File "C:\Users\h.decure\Anaconda3\lib\site-packages\gradio\routes.py", line 384, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\Users\h.decure\Anaconda3\lib\site-packages\gradio\blocks.py", line 1024, in process_api
    result = await self.call_function(
  File "C:\Users\h.decure\Anaconda3\lib\site-packages\gradio\blocks.py", line 850, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\h.decure\Anaconda3\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "C:\Users\h.decure\Anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "C:\Users\h.decure\Anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run(func, *args)
  File "C:\Users\h.decure\Anaconda3\lib\site-packages